# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [1]:
import sys

print(sys.executable)
print(sys.path)

/opt/miniconda3/envs/ml/bin/python
['/Users/chaklader/Documents/Education/Udacity/Deep_Learning/Projects/3_movie_sentiment_analysis', '/usr/local/Cellar/apache-spark/3.1.2/libexec/python', '/Users/chaklader/Documents/Education/Udacity/Deep_Learning/Projects/3_movie_sentiment_analysis/$', '/opt/miniconda3/envs/ml/lib/python312.zip', '/opt/miniconda3/envs/ml/lib/python3.12', '/opt/miniconda3/envs/ml/lib/python3.12/lib-dynload', '', '/opt/miniconda3/envs/ml/lib/python3.12/site-packages', '/opt/miniconda3/envs/ml/lib/python3.12/site-packages/setuptools/_vendor']


In [5]:
import pandas as pd
from transformers import pipeline

print(f"Transformers version: {transformers.__version__}")

Transformers version: 4.56.1


### Get data from `.csv` files and then preprocess data

In [9]:
def preprocess_data() -> pd.DataFrame:
    """
    Purpose:
        Reads movie data from CSV files in three languages (English, French, Spanish) and 
        combines them into a single DataFrame with language identification for multilingual 
        sentiment analysis processing.
    
    Parameters:
        None - Function reads from predefined CSV file paths in the data/ directory
    
    Process Flow:
        1. Define consistent column names for all CSV files
        2. Read English, French, and Spanish movie review CSV files
        3. Add 'Original Language' column to each DataFrame (en/fr/sp)
        4. Concatenate all DataFrames with reset index
        5. Return combined DataFrame ready for translation and analysis
    
    Outputs:
        pd.DataFrame: Combined dataset with columns ['Title', 'Year', 'Synopsis', 'Review', 'Original Language']
                     Contains 30 rows (10 movies per language) for sentiment analysis pipeline
    
    Example:
        >>> df = preprocess_data()
        >>> print(df.shape)
        (30, 5)
        >>> print(df['Original Language'].value_counts())
        en    10
        fr    10
        sp    10
    """
    # Read CSV files with consistent column names
    column_names = ['Title', 'Year', 'Synopsis', 'Review']
    
    df_eng = pd.read_csv("data/movie_reviews_en.csv", names=column_names, header=0)
    df_fr = pd.read_csv("data/movie_reviews_fr.csv", names=column_names, header=0)
    df_sp = pd.read_csv("data/movie_reviews_sp.csv", names=column_names, header=0)
    
    df_eng['Original Language'] = 'en'
    df_fr['Original Language'] = 'fr'
    df_sp['Original Language'] = 'sp'
    
    df_combined = pd.concat([df_eng, df_fr, df_sp], ignore_index=True)
    
    return df_combined

df = preprocess_data()

In [7]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
18,Les Visiteurs en Amérique,2000,Dans cette suite de la comédie française Les V...,"""Le film est une perte de temps totale. Les bl...",fr
12,Amélie,2001,Cette comédie romantique raconte l'histoire d'...,"""Amélie est un film absolument charmant qui vo...",fr
25,Águila Roja,(2009-2016),Esta serie de televisión española sigue las av...,"""Águila Roja es una serie aburrida y poco inte...",sp
29,El Incidente,2014,"En esta película de terror mexicana, un grupo ...","""El Incidente es una película aburrida y sin s...",sp
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",en
8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of...",en
15,Le Dîner de Cons,1998,Le film suit l'histoire d'un groupe d'amis ric...,"""Je n'ai pas aimé ce film du tout. Le concept ...",fr
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",en
21,La Casa de Papel,(2017-2021),Esta serie de televisión española sigue a un g...,"""La Casa de Papel es una serie emocionante y a...",sp
11,Intouchables,2011,Ce film raconte l'histoire de l'amitié improba...,"""Intouchables est un film incroyablement touch...",fr


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [10]:
"""
Purpose:
    Initializes pre-trained Marian neural machine translation models and tokenizers
    for French-to-English and Spanish-to-English translation. Sets up the translation
    pipeline components needed for multilingual movie review processing.

Parameters:
    None - Uses predefined model names from Helsinki-NLP repository

Process Flow:
    1. Define model names for French-English and Spanish-English translation
    2. Load pre-trained MarianMTModel instances from HuggingFace Hub
    3. Load corresponding MarianTokenizer instances for text preprocessing
    4. Store models and tokenizers in variables for translation pipeline use

Outputs:
    fr_en_model: MarianMTModel for French to English translation
    es_en_model: MarianMTModel for Spanish to English translation  
    fr_en_tokenizer: MarianTokenizer for French text preprocessing
    es_en_tokenizer: MarianTokenizer for Spanish text preprocessing

Example:
    >>> print(fr_en_model_name)
    "Helsinki-NLP/opus-mt-fr-en"
    >>> french_text = "Bonjour le monde"
    >>> inputs = fr_en_tokenizer(french_text, return_tensors="pt")
    >>> outputs = fr_en_model.generate(**inputs)
"""

from transformers import MarianMTModel, MarianTokenizer

fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"

fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)

fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)

/opt/miniconda3/envs/ml/lib/python3.12/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
def translate(text: str, model, tokenizer) -> str:
    """
    Purpose:
        Translates text from source language to English using pre-trained Marian MT models
        from HuggingFace. Handles text preprocessing and tokenization for neural machine
        translation with proper truncation and padding.
    
    Parameters:
        text (str): Single text string to translate (review or synopsis content)
        model: Pre-trained MarianMTModel for translation (e.g., Helsinki-NLP/opus-mt-fr-en)
        tokenizer: Corresponding MarianTokenizer for text preprocessing and decoding
    
    Process Flow:
        1. Validate and convert input text to string format
        2. Tokenize text with padding, truncation (max 512 tokens) and PyTorch tensors
        3. Generate translation using the model's generate() method
        4. Decode output tokens to readable text, skipping special tokens
        5. Return clean translated text string
    
    Outputs:
        str: Translated text in English, cleaned of special tokens and formatting
             Ready for sentiment analysis processing
    
    Example:
        >>> fr_text = "Ce film est magnifique et très émouvant"
        >>> translated = translate(fr_text, fr_en_model, fr_en_tokenizer)
        >>> print(translated)
        "This film is beautiful and very moving"
    """
    if not isinstance(text, str):
        text = str(text)


    """
    Tokenizes input text and converts it to PyTorch tensors for model processing.

    Parameters breakdown:
        text: The input string to be tokenized and encoded
        return_tensors="pt": Specifies output format as PyTorch tensors (alternative: "tf" for TensorFlow, "np" for NumPy)
                            Required because neural models expect tensor inputs, not raw text
        padding=True: Automatically pads shorter sequences to match batch dimensions
                    Ensures consistent input shape when processing multiple texts
        truncation=True: Cuts off text exceeding max_length to prevent memory overflow
                        Essential for handling variable-length inputs safely
        max_length=512: Maximum sequence length in tokens (common limit for transformer models)
                        Balances memory usage with context preservation

    Returns:
        dict: Contains 'input_ids' (token indices) and 'attention_mask' (padding indicators)
            Ready for direct input to the translation model
    """
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    """
    Generates translated text using the pre-trained translation model.

    Parameter breakdown:
        **inputs: Unpacks the dictionary returned by tokenizer into keyword arguments
                Equivalent to: model.generate(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
                The ** operator spreads dictionary keys as parameter names automatically
        
    Process:
        Uses the model's generate() method which performs beam search or sampling
        to produce the most likely translation sequence token by token
        
    Returns:
        torch.Tensor: Raw output token IDs representing the translated text
                    Shape: [batch_size, sequence_length] containing integer token indices
    """
    outputs = model.generate(**inputs)


    """
    Converts generated token IDs back to human-readable text string.

    Parameter breakdown:
        outputs[0]: Selects first (and only) sequence from the batch dimension
                    Since we're translating one text, we take index 0 from the batch
        skip_special_tokens=True: Removes model-specific tokens like <pad>, <eos>, <bos>
                                Produces clean output without internal formatting markers

    Process:
        Maps integer token IDs back to their corresponding text representations
        using the tokenizer's vocabulary and decoding rules
        
    Returns:
        str: Clean, readable translated text ready for downstream processing
            Free of special tokens and model artifacts
    """    
    translated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return translated

/opt/miniconda3/envs/ml/lib/python3.12/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [6]:
fr_mask = df['Original Language'] == 'fr'
sp_mask = df['Original Language'] == 'sp'

#  previous version that I have changed 

# for idx in df[fr_mask].index:
#     df.at[idx, 'Review'] = translate(df.at[idx, 'Review'], fr_en_model, fr_en_tokenizer)
#     df.at[idx, 'Synopsis'] = translate(df.at[idx, 'Synopsis'], fr_en_model, fr_en_tokenizer)

# for idx in df[sp_mask].index:
#     # Translate review
#     df.at[idx, 'Review'] = translate(df.at[idx, 'Review'], es_en_model, es_en_tokenizer)
#     # Translate synopsis
#     df.at[idx, 'Synopsis'] = translate(df.at[idx, 'Synopsis'], es_en_model, es_en_tokenizer)


translation_configs = {
    'fr': (fr_mask, fr_en_model, fr_en_tokenizer),
    'sp': (sp_mask, es_en_model, es_en_tokenizer)
}

for lang, (mask, model, tokenizer) in translation_configs.items():
    for col in ['Review', 'Synopsis']:
        df.loc[mask, col] = df.loc[mask, col].apply(
            lambda x: translate(x, model, tokenizer)
        )

In [7]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
14,Le Fabuleux Destin d'Amélie Poulain,2001,This romantic comedy tells the story of Amélie...,"""The Fabulous Destiny of Amélie Poulain is an ...",fr
29,El Incidente,2014,"In this Mexican horror film, a group of people...","""The Incident is a boring and frightless film ...",sp
5,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",en
23,El Laberinto del Fauno,2006,"During the Spanish postwar period, Ofelia (Iva...","""The Labyrinth of Fauno is a fascinating and e...",sp
25,Águila Roja,(2009-2016),This Spanish television series follows the adv...,"""Red Eagle is a boring and uninteresting serie...",sp
8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of...",en
28,Torrente: El brazo tonto de la ley,1998,"In this Spanish comedy, a corrupt cop (played ...","""Torrente is a vulgar and offensive film that ...",sp
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",en
24,Amores perros,2000,Three stories intertwine in this Mexican film:...,"""Amores dogs is an intense and moving film tha...",sp
20,Roma,2018,Cleo (Yalitza Aparicio) is a young domestic wo...,"""Rome is a beautiful and moving film that pays...",sp


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [8]:
import torch

# MPS for Apple machine 
device = (
    "mps" if torch.backends.mps.is_available()
    else "cuda" if torch.cuda.is_available()
    else "cpu"
)

model_name = "distilbert-base-uncased-finetuned-sst-2-english"
sentiment_classifier = pipeline(
    "sentiment-analysis", 
    model=model_name,
    device=device 
)

def analyze_sentiment(text, classifier):
    """
    Function to perform sentiment analysis on text
    """
    result = classifier(text)[0]
    # Return "Positive" if POSITIVE label and score > 0.5, else "Negative"
    return "Positive" if result['label'] == 'POSITIVE' and result['score'] > 0.5 else "Negative"


Device set to use mps


In [10]:
import os
os.makedirs("result", exist_ok=True)

def clean_text(text):
    """Clean and validate text data"""
    if pd.isna(text) or text == 'nan' or not isinstance(text, str):
        return ""
    return text.strip('"')  # Remove quote marks that appear in reviews


df['Review'] = df['Review'].apply(clean_text)
df = df[df['Review'] != ""]

df['Review Sentiment'] = df['Review'].apply(lambda x: analyze_sentiment(x, sentiment_classifier))

df = df[['Title', 'Year', 'Synopsis', 'Review', 'Review Sentiment', 'Original Language']]

df = df.head(30)

if len(df) != 30:
    print(f"Warning: Expected 30 rows but got {len(df)} rows")

required_columns = ['Title', 'Year', 'Synopsis', 'Review', 'Review Sentiment', 'Original Language']
missing_columns = [col for col in required_columns if col not in df.columns]
if missing_columns:
    print(f"Warning: Missing required columns: {missing_columns}")

# Save to CSV
df.to_csv("result/reviews_with_sentiment.csv", index=False)

In [10]:
df.sample(10)

,Title,Year,Synopsis,Review,Review Sentiment,Original Language
23,El Laberinto del Fauno,2006,"During the Spanish postwar period, Ofelia (Iva...",The Labyrinth of Fauno is a fascinating and em...,Positive,sp
28,Torrente: El brazo tonto de la ley,1998,"In this Spanish comedy, a corrupt cop (played ...",Torrente is a vulgar and offensive film that a...,Negative,sp
5,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...",Boring and too long. Nothing like the original...,Negative,en
24,Amores perros,2000,Three stories intertwine in this Mexican film:...,Amores dogs is an intense and moving film that...,Positive,sp
8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"Dull and pointless, with none of the magic of ...",Negative,en
14,Le Fabuleux Destin d'Amélie Poulain,2001,This romantic comedy tells the story of Amélie...,The Fabulous Destiny of Amélie Poulain is an a...,Positive,fr
29,El Incidente,2014,"In this Mexican horror film, a group of people...",The Incident is a boring and frightless film t...,Negative,sp
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...",The Shawshank Redemption is an inspiring tale ...,Positive,en
10,La La Land,2016,This musical tells the story of a budding actr...,The Land is an absolutely beautiful film with ...,Positive,fr
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,It was difficult to sit through the whole thin...,Negative,en


In [11]:
df.to_csv("result/reviews_with_sentiment.csv", index=False)